<a href="https://colab.research.google.com/github/darknight11345/turtlebot4/blob/main/Text24_01_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:

!nvidia-smi

Fri Jan 24 20:55:34 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0              29W /  70W |   3843MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
pwd

'/content'

In [31]:
#setup of dataset
import torch
print (torch.cuda.is_available())

True


In [32]:
!pip install simpletransformers

  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
!pip install fsspec==2024.10.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.2.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2024.10.0 which is incompatible.


In [73]:
import pandas as pd
data = pd.read_csv("ner_dataset_with_head_towards_and_noise_23_01_25.csv",encoding="latin1" )

In [74]:
data.head(30)

,Sentence_ID,Word,POS,NER
0,1,Bring,VERB,ACTION
1,1,Ibuprofen,PROPN,O
2,1,to,ADP,O
3,1,Reception,NOUN,LOC
4,2,Navigate,VERB,ACTION
5,2,to,ADP,O
6,2,Room,NOUN,Beg-LOC
7,2,363,NUM,Ins-LOC
8,2,with,ADP,O
9,2,Aspirin,NOUN,O


In [75]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [76]:
data.rename(columns={"Sentence_ID":"sentence_id","Word":"words","NER":"labels"}, inplace =True)

In [77]:
data["labels"] = data["labels"].str.upper()
data["words"] = data["words"].str.lower()

In [78]:
X= data[["sentence_id","words"]]
Y =data["labels"]

In [79]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [80]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [81]:
test_data

,sentence_id,words,labels
746,158,to,O
9114,1903,room,O
3044,638,room,BEG-LOC
8502,1778,with,O
4134,866,to,O
...,...,...,...
922,195,to,O
9009,1880,to,O
388,81,room,O
8593,1797,move,ACTION


In [82]:
train_data

,sentence_id,words,labels
9522,1989,navigate,ACTION
3936,824,300,INS-LOC
9322,1948,penicillin,O
9435,1971,to,O
2073,434,tylenol,O
...,...,...,...
7406,1547,icu,LOC
5710,1192,room,O
647,137,tylenol,O
6508,1361,bring,ACTION


In [ ]:
#Model Training

In [48]:

!pip install torch  --index-url https://download.pytorch.org/whl/cu124


Looking in indexes: https://download.pytorch.org/whl/cu124


In [49]:


from simpletransformers.ner import NERModel,NERArgs

In [83]:
label = data["labels"].unique().tolist()
label

['ACTION', 'O', 'LOC', 'BEG-LOC', 'INS-LOC']

In [85]:
args = NERArgs()
args.num_train_epochs = 11
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32
args.gradient_accumulation_steps = 4  # Adjust this to your needs
args.max_seq_length = 128  # Adjust this according to your data
args.evaluate_during_training: True


In [86]:
model = NERModel('bert', 'bert-base-uncased',labels=label,args =args, use_cuda=True)
#model = NERModel("bert", "outputs", use_cuda=False) # to use pretained model

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [87]:
#model.train_model(train_data,eval_data = test_data,acc=accuracy_score)
output_dir = "./best_model"
#output_dir = "./best_model_pretained_model"
model.train_model(train_data, eval_data=test_data, acc=accuracy_score, save_best_model=True, output_dir=output_dir)

model.save_model("./text_trained_model_bert")

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/11 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/ner/ner_model.py:758: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 11:   0%|          | 0/63 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/ner/ner_model.py:782: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


...

Running Epoch 7 of 11:   0%|          | 0/63 [00:00<?, ?it/s]

Running Epoch 8 of 11:   0%|          | 0/63 [00:00<?, ?it/s]

Running Epoch 9 of 11:   0%|          | 0/63 [00:00<?, ?it/s]

Running Epoch 10 of 11:   0%|          | 0/63 [00:00<?, ?it/s]

Running Epoch 11 of 11:   0%|          | 0/63 [00:00<?, ?it/s]

In [88]:
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/41 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/ner/ner_model.py:1303: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ACTION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: BEG-LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: INS-LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [89]:
result

{'eval_loss': 0.18057744994350686,
 'precision': 0.9602194787379973,
 'recall': 0.9333333333333333,
 'f1_score': 0.9465855307640297}

In [92]:
prediction, model_output = model.predict(["  tablets deliver to room 111"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [93]:
prediction

[[{'tablets': 'O'},
  {'deliver': 'ACTION'},
  {'to': 'O'},
  {'room': 'O'},
  {'111': 'O'}]]

In [24]:
model_output

[[{'deliver': [[-0.2864, 3.508, -0.999, -1.131, -0.3145]]},
  {'deliver': [[-0.02869, 3.387, -1.069, -1.142, -0.4688]]},
  {'tablets': [[-0.2761, 3.182, -0.5415, -0.9346, -0.7354]]},
  {'to': [[-0.756, 4.184, -1.301, -1.453, -0.8877]]},
  {'room': [[-0.602, 3.512, -0.944, -0.7305, -0.704]]},
  {'102': [[-0.809, 1.012, -1.246, 2.492, -0.6436]]}]]

In [ ]:
#inference

# Load the best saved model for inference
#model1 = NERModel("bert", "./best_model", use_cuda=False)


In [102]:
#model1 = NERModel("bert", "./outputs", use_cuda=False)

model1 = NERModel("bert", "./best_model/checkpoint-165-epoch-11", use_cuda=True)

In [103]:
prediction1, model_output = model1.predict(["deliver paracetamol to room 102 "])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/ner/ner_model.py:1643: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


In [104]:
prediction1

[[{'deliver': 'ACTION'},
  {'paracetamol': 'O'},
  {'to': 'O'},
  {'room': 'O'},
  {'102': 'O'}]]